In [1]:
# =============================================
# 1️⃣ Install / Import Packages
# =============================================
!pip install --pre azure-ai-projects>=2.0.0b1
!pip install azure-identity

import os
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient

In [66]:
from azure.identity import DefaultAzureCredential

cred = DefaultAzureCredential()
token = cred.get_token("https://management.azure.com/.default")
print("Token acquired:", token.token[:20], "...")  # prints first 20 chars

DefaultAzureCredential failed to retrieve a token from the included credentials.
Attempted credentials:
	EnvironmentCredential: EnvironmentCredential authentication unavailable. Environment variables are not fully configured.
Visit https://aka.ms/azsdk/python/identity/environmentcredential/troubleshoot to troubleshoot this issue.
	WorkloadIdentityCredential: WorkloadIdentityCredential authentication unavailable. The workload options are not fully configured. See the troubleshooting guide for more information: https://aka.ms/azsdk/python/identity/workloadidentitycredential/troubleshoot. Missing required arguments: 'tenant_id', 'client_id', 'token_file_path'.
	ManagedIdentityCredential: ManagedIdentityCredential authentication unavailable, no response from the IMDS endpoint. Token request error: (invalid_request) Identity not found
	SharedTokenCacheCredential: SharedTokenCacheCredential authentication unavailable. No accounts were found in the cache.
	VisualStudioCodeCredential: VisualSt

ClientAuthenticationError: DefaultAzureCredential failed to retrieve a token from the included credentials.
Attempted credentials:
	EnvironmentCredential: EnvironmentCredential authentication unavailable. Environment variables are not fully configured.
Visit https://aka.ms/azsdk/python/identity/environmentcredential/troubleshoot to troubleshoot this issue.
	WorkloadIdentityCredential: WorkloadIdentityCredential authentication unavailable. The workload options are not fully configured. See the troubleshooting guide for more information: https://aka.ms/azsdk/python/identity/workloadidentitycredential/troubleshoot. Missing required arguments: 'tenant_id', 'client_id', 'token_file_path'.
	ManagedIdentityCredential: ManagedIdentityCredential authentication unavailable, no response from the IMDS endpoint. Token request error: (invalid_request) Identity not found
	SharedTokenCacheCredential: SharedTokenCacheCredential authentication unavailable. No accounts were found in the cache.
	VisualStu

In [ ]:


# =============================================
# 2️⃣ Define UKEF Agent (Live Azure Option)
# =============================================
USE_AZURE_AGENT = True  # Set False to use simulated UKEF agent

if USE_AZURE_AGENT:
    myEndpoint = ""
    project_client = AIProjectClient(
        endpoint=myEndpoint,
        credential=DefaultAzureCredential(),
    )
    myAgent = "UKEF1"
    agent = project_client.agents.get(agent_name=myAgent)
    print(f"Retrieved agent: {agent.name}")
    openai_client = project_client.get_openai_client()

    def ask_ukef_agent(question):
        try:
            resp = openai_client.responses.create(
                input=[{"role": "user", "content": question}],
                extra_body={"agent": {"name": agent.name, "type": "agent_reference"}},
            )
            answer = resp.output_text
            if not answer.strip():
                return "Simulated fallback: Coverage applies to low-risk projects up to £5M."
            return answer
        except Exception:
            return "Simulated fallback: Coverage applies to low-risk projects up to £5M."
else:
    def ask_ukef_agent(question):
        return "Simulated fallback: Coverage applies to low-risk projects up to £5M."

# =============================================
# 3️⃣ Define PhD Agent (Simulated / Local RAG)
# =============================================
def ask_phd_agent(question):
    folder = "data_phd"
    if not os.path.exists(folder):
        os.makedirs(folder)
        with open(f"{folder}/doc1.txt", "w") as f:
            f.write("This is a sample PhD methodology document covering experiments and analysis.")
    
    docs = [open(f"{folder}/{f}").read() for f in os.listdir(folder) if f.endswith(".txt")]
    context = "\n".join(docs)
    return f"PhD Agent simulated answer:\nContext:\n{context}\nQuestion:\n{question}"

# =============================================
# 4️⃣ Define Finance Agent (Simulated)
# =============================================
def ask_finance_agent(question):
    folder = "data_finance"
    if not os.path.exists(folder):
        os.makedirs(folder)
        with open(f"{folder}/doc1.txt", "w") as f:
            f.write("Finance Agent docs: budgets, risk analysis, and cash flows.")
    
    docs = [open(f"{folder}/{f}").read() for f in os.listdir(folder) if f.endswith(".txt")]
    context = "\n".join(docs)
    return f"Finance Agent simulated answer:\nContext:\n{context}\nQuestion:\n{question}"

# =============================================
# 5️⃣ Orchestrator Agent
# =============================================
def orchestrator_agent(question):
    """
    Orchestrator = agent itself
    Routes queries to child agents and aggregates responses
    """
    q = question.lower()
    if "ukef" in q or "coverage" in q or "mandate" in q:
        return ask_ukef_agent(question)
    elif "phd" in q or "research" in q or "methodology" in q:
        return ask_phd_agent(question)
    elif "finance" in q or "budget" in q:
        return ask_finance_agent(question)
    else:
        # Call all agents and aggregate
        resp1 = ask_ukef_agent(question)
        resp2 = ask_phd_agent(question)
        resp3 = ask_finance_agent(question)
        return f"{resp1}\n---\n{resp2}\n---\n{resp3}"

# =============================================
# 6️⃣ Example Queries / Demo
# =============================================
example_questions = [
    "Explain coverage rules for UKEF projects.",
    "Summarize key points of my PhD methodology.",
    "How does the No Net Cost mandate apply?",
    "What is the maximum project risk we can cover?",
    "Provide budget insights for low-risk projects."
]

for q in example_questions:
    print("Q:", q)
    print("A:", orchestrator_agent(q))
    print("-"*80)


Retrieved agent: UKEF1
Q: Explain coverage rules for UKEF projects.
A: Simulated fallback: Coverage applies to low-risk projects up to £5M.
--------------------------------------------------------------------------------
Q: Summarize key points of my PhD methodology.
A: PhD Agent simulated answer:
Context:
This is a sample PhD methodology document covering experiments and analysis.
Question:
Summarize key points of my PhD methodology.
--------------------------------------------------------------------------------
Q: How does the No Net Cost mandate apply?
A: The 'No Net Cost' mandate requires UKEF to operate so that its long-term financial operations do not create a loss for HM Treasury. (Source: UKEF 'No Net Cost' mandate)
--------------------------------------------------------------------------------
Q: What is the maximum project risk we can cover?
A: Not found in the documents.
---
PhD Agent simulated answer:
Context:
This is a sample PhD methodology document covering experiments

In [8]:
!pip install semantic-kernel azure-identity openai


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 910.9/910.9 kB 28.7 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 948.6/948.6 kB 40.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 39.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 49.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 MB 40.8 MB/s  0:00:01m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 48.1 MB/s  0:00:006m0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 39.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.0/35.0 MB 47.7 MB/s  0:00:006m0:00:0100:01
  Created wheel for pybars4: filename=pybars4-0.9.13-py3-none-any.whl size=14418 sha256=

In [ ]:
!pip install semantic-kernel azure-identity

In [28]:
import semantic_kernel as sk
from semantic_kernel.functions import kernel_function
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from azure.identity import DefaultAzureCredential

kernel = sk.Kernel()


In [30]:
# --- Define Native Plugins ---

class UKEFPlugin:
    @kernel_function(
        name="UKEFAgent",
        description="Provides policy coverage details and financial limits for UKEF projects."
    )
    def get_coverage(self, query: str) -> str:
        # This is where your CSV-reading logic would live
        return "UKEF policy coverage applies to low-risk projects up to £5M."

class PhDPlugin:
    @kernel_function(
        name="PhDAgent",
        description="Handles academic methodology, research experiments, and thesis evaluation."
    )
    def get_research(self, query: str) -> str:
        return "PhD agent: Specialized in methodology and systematic evaluation."

# --- Register Plugins ---
kernel.add_plugin(UKEFPlugin(), plugin_name="UKEF")
kernel.add_plugin(PhDPlugin(), plugin_name="PhD")

KernelPlugin(name='PhD', description=None, functions={'PhDAgent': KernelFunctionFromMethod(metadata=KernelFunctionMetadata(name='PhDAgent', plugin_name='PhD', description='Handles academic methodology, research experiments, and thesis evaluation.', parameters=[KernelParameterMetadata(name='query', description=None, default_value=None, type_='str', is_required=True, type_object=<class 'str'>, schema_data={'type': 'string'}, include_in_function_choices=True)], is_prompt=False, is_asynchronous=False, return_parameter=KernelParameterMetadata(name='return', description='', default_value=None, type_='str', is_required=True, type_object=<class 'str'>, schema_data={'type': 'string'}, include_in_function_choices=True), additional_properties={}), invocation_duration_histogram=<opentelemetry.metrics._internal.instrument._ProxyHistogram object at 0x77f5ddf5dd60>, streaming_duration_histogram=<opentelemetry.metrics._internal.instrument._ProxyHistogram object at 0x77f5de781a60>, method=<bound method

In [42]:
from semantic_kernel.connectors.ai.function_choice_behavior import FunctionChoiceBehavior
from semantic_kernel.connectors.ai.open_ai import AzureChatPromptExecutionSettings

async def agentic_orchestrator(query: str):
    # Set behavior to automatically call functions (plugins)
    settings = AzureChatPromptExecutionSettings(
        function_choice_behavior=FunctionChoiceBehavior.Auto()
    )
    
    # We use invoke_prompt. The LLM sees the description of UKEFAgent and PhDAgent
    # and decides which one to call based on the user's intent.
    result = await kernel.invoke_prompt(
        function_name="Orchestrator",
        plugin_name="Main",
        prompt=query,
        settings=settings
    )
    return result


In [ ]:
# Option 1: The 'Clean Slate' approach (Recommended for Notebooks)
#kernel.remove_all_services()

# Now add your fixed service with the token_provider
#kernel.add_service(azure_chat)

# Option 2: The 'Overwrite' approach
# kernel.add_service(azure_chat, overwrite=True)

In [43]:
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

# 1. Create the credential
cred = DefaultAzureCredential()

# 2. CREATE A TOKEN PROVIDER with the correct 'Cognitive Services' scope
# This is the "Magic Sauce" that fixes the 401 Audience error
token_provider = get_bearer_token_provider(
    cred, 
    "https://cognitiveservices.azure.com/.default"
)

# 3. Initialize AzureChatCompletion using the token_provider
azure_chat = AzureChatCompletion(
    deployment_name="gpt-4o",
    endpoint=myEndpoint,
    ad_token_provider=token_provider # Use this instead of passing 'cred' directly
)

kernel.add_service(azure_chat)

In [46]:
import semantic_kernel as sk

# 1. Complete Kernel Reset
kernel = sk.Kernel() 

# 2. Add the service with the fixed token provider
# (Make sure azure_chat is already defined with ad_token_provider)
kernel.add_service(azure_chat)

# 3. Re-register Plugins (This overwrites anything old)
# We don't need to "remove" them because we made a 'new' kernel above
kernel.add_plugin(UKEFPlugin(), plugin_name="UKEF")
kernel.add_plugin(PhDPlugin(), plugin_name="PhD")

print("✅ Kernel initialized from scratch. No conflicts!")

✅ Kernel initialized from scratch. No conflicts!


In [48]:
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

# 1. Create the credential as before
cred = DefaultAzureCredential()

# 2. CREATE A TOKEN PROVIDER with the specific scope for AI services
# This specifically fixes the "audience is incorrect" error.
token_provider = get_bearer_token_provider(
    cred, 
    "https://cognitiveservices.azure.com/.default"
)

# 3. Update your AzureChatCompletion to use 'ad_token_provider'
azure_chat = AzureChatCompletion(
    deployment_name="gpt-4o",
    endpoint=myEndpoint,
    ad_token_provider=token_provider  # Use the provider, not the raw credential
)

# 4. Clean and re-add the service to avoid the "Already Exists" error
kernel.remove_all_services()
kernel.add_service(azure_chat)

In [51]:
import semantic_kernel as sk
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

# --- 1. CLEAN YOUR ENDPOINT ---
# Ensure myEndpoint is ONLY: https://your-resource-name.openai.azure.com/
# (No trailing slashes, no /openai/ path)
clean_endpoint = myEndpoint.split("/openai/")[0].strip("/")

# --- 2. SETUP IDENTITY ---
token_provider = get_bearer_token_provider(
    DefaultAzureCredential(), 
    "https://cognitiveservices.azure.com/.default"
)

# --- 3. INITIALIZE SERVICE WITH STABLE API VERSION ---
azure_chat = AzureChatCompletion(
    deployment_name="gpt-4o",
    endpoint=clean_endpoint,
    ad_token_provider=token_provider,
    api_version="2024-05-01-preview" # Using a guaranteed stable version
)

# --- 4. CLEAN RESET ---
kernel = sk.Kernel()
kernel.add_service(azure_chat)

# --- 5. RE-REGISTER PLUGINS ---
# Ensure these classes are defined in your notebook!
kernel.add_plugin(UKEFPlugin(), plugin_name="UKEF")
kernel.add_plugin(PhDPlugin(), plugin_name="PhD")

print("✅ System fully reset with cleaned endpoint and stable API version.")

✅ System fully reset with cleaned endpoint and stable API version.


In [ ]:
import requests
# Test if your environment can even "see" the endpoint
try:
    response = requests.get(clean_endpoint)
    print(f"Network Check: Connection successful (Status: {response.status_code})")
except Exception as e:
    print(f"❌ Network Check Failed: Your firewall or VPN is blocking the connection. Error: {e}")

In [54]:
# The scope MUST be cognitiveservices, not ai.azure.com
token_provider = get_bearer_token_provider(
    DefaultAzureCredential(), 
    "https://cognitiveservices.azure.com/.default"
)

In [61]:
token_info = cred.get_token("https://cognitiveservices.azure.com/.default")


In [ ]:
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

kernel = Kernel()

kernel.add_service(
    AzureChatCompletion(
        deployment_name="gpt-4o",  # must match the deployment in abphd
        endpoint="/",  # ✅ base endpoint
        api_key=""  # ✅ use Key1
    ),
    overwrite=True
)


In [107]:
existing_agents = [name for name in ["agent_ukef", "agent_phd"] if name in globals()]
print("Existing agents:", existing_agents)

Existing agents: ['agent_ukef', 'agent_phd']


In [ ]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, OpenAIPromptExecutionSettings
from semantic_kernel.contents import ChatHistory

kernel = sk.Kernel()

# Updated for gpt-4o-mini
chat_service = AzureChatCompletion(
    deployment_name="gpt-4o-mini",  # Make sure this matches your Azure Deployment Name
    endpoint="",
    api_key="",
    api_version="2024-08-01-preview" # Updated to a version that fully supports mini's features
)

kernel.add_service(chat_service)

async def test_mini_chat():
    history = ChatHistory()
    history.add_user_message("Briefly explain why the UKEF 'No Net Cost' mandate is a 'solvency-based constraint' for a PhD methodology.")
    
    # We use a lower temperature (0.3) for more "academic" and precise results
    settings = OpenAIPromptExecutionSettings(max_tokens=500, temperature=0.3)
    
    try:
        response = await chat_service.get_chat_message_contents(
            chat_history=history,
            settings=settings
        )
        print("🤖 GPT-4o-Mini Response:\n")
        print(response[0].content)
    except Exception as e:
        print(f"❌ Error: {e}")

await test_mini_chat()

🤖 GPT-4o-Mini Response:

The UKEF (UK Export Finance) 'No Net Cost' mandate is considered a 'solvency-based constraint' for a PhD methodology because it requires that any financial support or guarantees provided by UKEF must not result in a net loss to the government. This means that the financial viability and sustainability of projects must be assessed in a way that ensures the expected revenues or benefits will at least cover the costs associated with the support provided.

In the context of a PhD methodology, this constraint necessitates a rigorous evaluation of the economic impacts, risks, and financial models associated with the research. Researchers must demonstrate that their proposed projects are not only feasible but also financially sound, ensuring that they can achieve a positive net outcome. This focus on solvency influences the design of the research, the selection of case studies, and the analytical frameworks used, as the methodology must align with the principles of fi

In [123]:
async def semantic_phd_chat():
    history = ChatHistory()
    
    # 1. THE SEMANTIC "PERSONA": This makes it a specialized agent
    history.add_system_message("""
    You are the 'UKEF-PhD Bridge Agent'. Your goal is to help the user integrate 
    financial mandates into academic research methodology. 
    
    CRITICAL CONSTRAINTS:
    - Focus on the 'No Net Cost' mandate as a 'solvency-based constraint'.
    - Use formal academic language.
    - References to prioritize: Koomey (2011) for efficiency and Grootendorst (2022).
    """)
    
    # 2. THE USER QUERY
    history.add_user_message("Explain the No Net Cost mandate as a solvency-based constraint.")
    
    settings = OpenAIPromptExecutionSettings(max_tokens=800, temperature=0.2) # Low temp for precision
    
    try:
        response = await chat_service.get_chat_message_contents(
            chat_history=history,
            settings=settings
        )
        print("🏛️ SEMANTIC AGENT RESPONSE:\n")
        print(response[0].content)
    except Exception as e:
        print(f"❌ Error: {e}")

await semantic_phd_chat()

🏛️ SEMANTIC AGENT RESPONSE:

The No Net Cost mandate serves as a critical solvency-based constraint within the context of financial management and economic policy, particularly in the realm of public financing and investment. This mandate stipulates that any financial initiative or project must be designed and executed in such a manner that it does not incur a net cost to the public treasury. In essence, it requires that the financial outlays associated with a project be offset by corresponding revenues or savings, thereby ensuring that the fiscal integrity of the governing body remains intact.

From a methodological perspective, the No Net Cost mandate necessitates a rigorous assessment of both the costs and benefits associated with any proposed initiative. This aligns with the principles articulated by Koomey (2011), who emphasizes the importance of efficiency in resource allocation and the need for a comprehensive evaluation of the economic implications of energy-related investments